In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import GroupKFold
from sklearn.utils import shuffle
from six.moves import cPickle

In [2]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline

In [3]:
X_train = pd.read_hdf('X_train_w32.h5')
y_train = pd.read_hdf('y_train_w32.h5')
X_test = pd.read_hdf('X_test_w32.h5')
X_test.drop('reordered', axis=1, inplace=True)

In [4]:
'''
#shuffle
X_train, y_train = shuffle(X_train, y_train, random_state=1969)
#add group K fold
K = 5
'''

'\n#shuffle\nX_train, y_train = shuffle(X_train, y_train, random_state=1969)\n#add group K fold\nK = 5\n'

In [5]:
# X_train.drop(['eval_set', 'user_id', 'product_id', 'order_id'], axis=1, inplace=True)

In [6]:
y_train = y_train.astype('int64')

In [7]:
y_train[:10]

0    1
1    1
2    0
3    0
4    1
5    0
6    0
7    0
8    1
9    1
Name: reordered, dtype: int64

In [8]:
y_train.unique()

array([1, 0], dtype=int64)

In [9]:
X = X_train
y = y_train

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
X_train.head()

,user_id,product_id,_up_order_count,_up_first_order_number,_up_last_order_number,_up_order_past_appears_mean,_up_order_past_appears_std,_up_average_cart_position,_up_std_cart_position,aisle_id,...,_up_order_since_last_order_normalize,_up_order_past_appears_mean_normalize,_up_order_past_appears_std_normalize,_up_order_rate_since_first_order,_up_order_past_appears_interval_mean,_up_order_past_appears_interval_std,_up_order_past_appears_interval_mean_normalize,_up_order_past_appears_interval_std_normalize,_up_order_expect_days_to_order,_up_order_expect_days_to_order_normalize
6980471,6363,13269,2,13,18,15.5,3.535534,9.5,4.949747,120,...,0.660377,0.861111,0.196419,0.048780,5.0,0.0,0.09434,0.0,-30.0,-0.566038
8228503,59675,12002,1,34,34,34.0,0.000000,16.0,0.000000,128,...,0.227273,1.000000,0.000000,0.090909,44.0,0.0,1.00000,0.0,34.0,0.772727
12236205,112228,15867,1,2,2,2.0,0.000000,12.0,0.000000,4,...,0.600000,1.000000,0.000000,0.250000,5.0,0.0,1.00000,0.0,2.0,0.400000
8239885,78485,36878,1,5,5,5.0,0.000000,18.0,0.000000,1,...,0.375000,1.000000,0.000000,0.250000,8.0,0.0,1.00000,0.0,5.0,0.625000
12401690,161890,45842,1,2,2,2.0,0.000000,12.0,0.000000,108,...,0.333333,1.000000,0.000000,0.500000,3.0,0.0,1.00000,0.0,2.0,0.666667


In [12]:
X_train.shape

(6779728, 60)

In [13]:
X_test.shape

(1694933, 60)

In [14]:
len(y_train)

6779728

In [15]:
len(y_test)

1694933

In [16]:
K = 5
groups = X_train.user_id
gkf = GroupKFold(n_splits=K).split(X=X_train, y=y_train, groups=groups)

In [17]:
gridParams = {'bagging_fraction': [0.8, 0.7, 0.6, 0.5],
              'lambda_l1': [0.1, 0.3, 1, 3, 10, 30],
              'lambda_l2': [0.3, 1, 3, 10, 30]}

In [18]:
lgb_est = lgb.LGBMModel(task='train',
                        boosting_type='gbdt',
                        learning_rate=0.1,
                        objective='binary',
                        metric=['binary_logloss', 'auc'],
                        num_leaves=128,
                        max_depth=12,
                        num_threads=2,
                        feature_fraction=0.75,
                        bagging_freq=5,
                        bagging_seed=1969)

In [19]:
model = GridSearchCV(estimator=lgb_est, param_grid=gridParams, cv=gkf, scoring='accuracy')

In [20]:
model.fit(X=X_train, y=y_train)

ValueError: Classification metrics can't handle a mix of binary and continuous targets

In [ ]:
lgb_train = lgb.Dataset(data=X_train, label=y_train)

In [ ]:
lgb_eval = lgb.Dataset(data=X_test, label=y_test, reference=lgb_train)

In [ ]:
params = {'task': 'train',
          'boosting_type': 'gbdt',
          'learning_rate': 0.1,
          'objective': 'binary',
          'metric': {'binary_logloss', 'auc'},
          'num_leaves': 128,
          'max_depth': 12,
          'num_threads': 2,
          'bagging_fraction': 0.9,
          'feature_fraction': 0.75,
          'bagging_freq': 5,
          'lambda_l1': 60,
          'lambda_l2': 30,
          'verbosity': -1,
          'bagging_seed':1969}

In [ ]:
gbm = lgb.train(params, lgb_train, valid_sets=lgb_eval)

In [ ]:
'''
#grid search
max_depth = [10]
num_leaves = [128]
feature_fraction = [0.75]
bagging_fraction = [0.8, 0.7, 0.6, 0.5]
lambda_l1 = [0.1, 0.3, 1, 3, 10, 30]
lambda_l2 = [0.3, 1, 3, 10, 30]
results = list()
for md in max_depth:
    params['max_depth'] = md
    for nl in num_leaves:
        params['num_leaves'] = nl
        for ff in feature_fraction:
            params['feature_fraction'] = ff
            for bf in bagging_fraction:
                params['bagging_fraction'] = bf
                for l1 in lambda_l1:
                    params['lambda_l1'] = l1
                    for l2 in lambda_l2:
                        params['lambda_l2'] = l2
                        param = [md, nl, ff, bf, l1, l2]
                        print(param)
                        bst = lgb.cv(params, train, num_boost_round=5000, metrics={'binary_logloss'}, folds=myGroups,
                                     early_stopping_rounds=20, verbose_eval=True, seed=777)
                        results.append((param, bst))
'''

In [ ]:
'''
f = open('gridsearch_result.save', 'wb')
cPickle.dump(results, f, protocol=cPickle.HIGHEST_PROTOCOL)
f.close()
'''
# steps = [('scaler', StandardScaler()), ('elasticnet', ElasticNet())]

In [ ]:
# pipeline = Pipeline(steps)

In [ ]:
# parameters = {'elasticnet__l1_ratio':np.linspace(0, 1, 30)}

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# model = GridSearchCV(estimator=pipeline, param_grid=parameters, cv=5)

In [ ]:
# model.fit(X_train, y_train)

In [ ]:
# y_pred = model.predict(X_test)